# <font color='blue'> Introduction to Libtorch</font>

---

<img src="https://www.learnopencv.com/wp-content/uploads/2020/09/c3-w15-libtorch.png" height="500">

---

We all know that PyTorch is a deep learning framework is written in C++ backend and wrapped in Python frontend.  
By this, what we mean is Pytorch functions like `torch.Tensor()`or modules like `torch.nn` implicitly call the underlying C++ code.

However, we also have something like a C++ wrapper (frontend) over the underlying C++ code, and we call it LibTorch (a library version).

At this point, we need to note that both the Libtorch and PyTorch are C++ and Python APIs, respectively, having a few commonly used functionalities like, 

- Tensor operations,

- Representing data,

- Creating neural networks,

- Optimization APIs,

- Data parallelization,

- etc.


The only difference is that we write C++ code in-case of the C++ frontend and write Python code in-case of the Python frontend.

We also need to note that this C++ frontend largely follows the design of the Python API.  
For example, a Pytorch code like `a = torch.randn(1,3,5,7)` would be `torch::Tensor a = torch::randn({1,3,5,7})` in C++.

In the following sections, we will learn some of the basic stuff we require to build a C++ training pipeline using LibTorch. 


```C++
// Including the necessary libraries..!!

#include <iostream>
#include <torch/torch.h>
using namespace torch::indexing;
```

##  <font color='green'> 1. Tensor Creation</font>


Tensors represent data. And this representation could be a set of random numbers or a set of consecutive numbers from 100 to 200 or a set of constant values as well.  

Generally, a Tensor representation follows some sort of a blueprint.  
And this blueprint could be something like `torch::function-name(function-specific-options, size, other-options)`.

There are some options for a Tensor such as its data-type, layout, and the device it lives in. Such options can be passed by using the namespace `torch::TensorOptions()`.


The following are some examples of the general Tensor functions.

```C++
//Create a Tensor of size [9,2,7,1]
torch::Tensor a = torch::randn({9,2,7,1});

//Create a Tensor with necessary datatype
torch::Tensor b = torch::arange(13, 43,  torch::TensorOptions().dtype(torch::kInt32));

//Reshape `b` to size-[2,5,3]
torch::Tensor c = b.view({2,-1,3}); // or b.view({-1,5,3}) or b.view({2,5,-1})
```

Sometimes, we need to operate over two Tensors. And, in this case, the Tensor operators or member functions available in Python frontend are also available in C++.


They can be found [here](https://pytorch.org/cppdocs/api/classat_1_1_tensor.html)


##  <font color='green'> 2. Tensor Indexing</font>

###  <font color='green'> 2.1 Getting a Tensor</font>


Indexing in the C++ works the same way as in Pytorch.
However, there are some changes in the code. 
Following is the table which tells us the equivalent code in Libtorch.

We don't have the `[]` operator in C++ frontend, so we need to use the Tensor's `.index()` member-function and then specify the indices.   
Note that the indexing modules are under the `torch::indexing` namespace, so it is better to use the `using namespace torch::indexing;` in the beginning to avoid writing lengthy code.


We shall consider an example in Pytorch and then convert it to the C++ code.


Consider `torch::Tensor A = torch::randn({9,5,7,4})`  . `A` being a random Tensor of size `[9,5,7,4]`

1. `A[:, 2:4, :, 1:3]` equivalenty in C++ is `A.index({Slice(), Slice(2,4), Slice(), Slice(1,3)})`.


2. `A[[1,3,5], 3:, 2, 3]` equivalently in C++ is `A.index({torch::tensor({1,3,5}), Slice(3,None), 2, 3})`.


3. `A[..., 3]` equivalently in C++ is `A.index({"...", 3})`


It would be a good exercise in guessing the resulting Tensor's size for all the three cases.

```C++
// let A be a random Tensor of size-[9,5,7,4]
torch::Tensor A = torch::randn({9,5,7,4});

// Then Libtorch equivalent of A[:,2:4,:,1:3] is shown below.
torch::Tensor subA1 = A.index({Slice(), Slice(2,4), Slice(), Slice(1,3)}); 
```

The above code demonstrated that how we can slice through a Tensor or get a sub-Tensor out of the original one using the `Tensor.index()` method.  

###  <font color='green'> 2.2 Setting a Tensor</font>

**Setting a sub-tensor with a Scalar**

This sub-section will learn about setting a sub-tensor with a scalar element or with another tensor.

We must be familiar with a code something like this. `A[...,3] = 77`.   
The above code sets all the sub-matrix elements resulting from `A[...,3]` to the value `77`.

We can do the exact way in Libtorch by using the Tensor's `.index_put()` member-function.

Concretely, the Libtorch way is `A.put_index_({"...",3}, 77)`.  
The first argument would be the sub-matrix, and the next argument will be a scalar value that has to be updated to the sub-matrix. 


```C++
//Setting a subtensor to a scalar value
A.index_put_({"...",3}, 77);
```

**Setting a sub-tensor with another tensor or a sub-tensor**

Sometimes, we also need to set a Tensor rather than just a Scalar. 
In Pytorch, if we had `X1 = torch.randn({4,5,6,7})` and `Y1 = torch.Tensor({3,2,2})`, and if we want to set particular submatrix of `X1` with `Y1`, we'd do something lke this.  
`X1[1:, 2:4, 0, 2:4] = Y1`.

The Libtorch way is shown below.

```C++
//setting X1's subtensor of size [3,2,2] with Y1's elements.
torch::Tensor X1 = torch::randn({4,5,6,7});
torch::Tensor Y1 = torch::randn({3,2,2});

//Pytorch equivalent is X1[1:, 2:4, 0, 2:4] = Y1
X1.index_put_({Slice(1,None), Slice(2,4), 0, Slice(2,4)}, Y1); 
```

```C++
//setting X2's subtensor of size-[3,2] with Y2's subtensor.
torch::Tensor X2 = torch::randn({4,5,6,7});
torch::Tensor Y2 = torch::randn({3,2,2});

//Pytorch equivalent is X2[1:, 2:4, 0, 2] = Y2[:,:,0]
X2.index_put_({Slice(1,None), Slice(2,4), 0, 2}, Y2.index({Slice(),Slice(),0}));
```

## <font color='green'>3. Creating a Neural Network</font>

**In PyTorch, we have two ways to create a neural network:**
1. The Modular way ie, creating a subclass inheriting from `torch.nn.Module`.   
2. The Sequential way ie, creating a sequential set of layers inside the `torch.nn.Sequential`.  


We have the same functionalities available in Libtorch as well, and in the following sub-sections, we'll see how we can create a Neural Network in Libtorch.  


###  <font color='green'>3.1 The Modular Way</font>

Before we dive into the Libtorch code, let's take a step back and see the components required to create a Neural network in Pytorch and then connect it with the Libtorch code.

We will consider a simple network with the following layers. `Convolution, Linear, MaxPool` and `BatchNorm`.  

Following is the network: 

``` Python
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.mxpool = nn.MaxPool2d(2,2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.mxpool(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
```

We need to note down a few things when we build a neural-network in Pytorch.
1. The network needs to inherit the parent `nn.Module` class
2. Create a constructor that consists of the layers we need.
3. Create a forward function to guide the flow of tensors from input to the output.

The Libtorch way is also the same, except for a few extra codes.
Let's first write the same neural network in Libtorch and see what the extra code is.

```C++
struct DemoNetImpl : torch::nn::Module
{   

    torch::nn::Linear fc1 , fc2 ;
    torch::nn::BatchNorm2d bn1 ;
    torch::nn::Conv2d conv1, conv2 ;
    torch::nn::MaxPool2d mxpool1 ;

    DemoNetImpl(int64_t N):
        conv1(torch::nn::Conv2dOptions(1, 32, 3).stride(1)),
        conv2(torch::nn::Conv2dOptions(32,64,3).stride(1)),
        fc1(torch::nn::Linear(9216, 128)),
        fc2(torch::nn::Linear(128, N)),
        bn1(torch::nn::BatchNorm2d(32)),
        mxpool1(torch::nn::MaxPool2dOptions(2).stride(2))
    {
        register_module("ConvLayer1", conv1);
        register_module("ConvLayer2", conv2);
        register_module("DenseLayer1", fc1);
        register_module("DenseLayer2", fc2);
        register_module("MaxPoolLayer", mxpool1);
        register_module("BatchnormLayer", bn1);

    }

    torch::Tensor forward(torch::Tensor x)
    {
        x = conv1(x);
        x = bn1(x);
        x = conv2(x);
        x = mxpool1(x);
        x = torch::flatten(x, 1);
        x = fc1(x);
        x = fc2(x);
        return x;
    }

}; 
TORCH_MODULE(DemoNet);
```

In line 1, we create the `DemoNet` struct inheriting the `torch::nn::Module` class.  
From line-4 to line-7, we declare the necessary layers. This is just like `self.conv1`; however, we declare it with the data-type it belongs to, such as `torch::nn::Conv2d conv1`.   

At this point, `conv1` is just a variable of type `Conv2d`; it has no information `(or options)` about the kernel-size or input-output channels.   
We assign such information in the constructor.    
From line-9 to line-15, create a constructor just like `self.__init__()` and assign the necessary attributes to those members.   
For example, `conv1(torch::nn::Conv2dOptions(1, 32, 3).stride(1))`.

We also see some unfamiliar stuff from line-17 to line-22. Well, the syntax reveals that we are REGISTERING THE MODULES.!  
What does it mean to register the modules? 

It just means that the weights for each module can be accessed and can be updated. That's it.  
If we call `DemoNet->parameters()`, we will be able to access the weights `(or learnable parameters)` for those modules.  
So, if we don't wrap a module under `register_modules()`, we simply won't access the respective modules' weights.  
Simple.!  


Once the constructor is defined, we have the `forward()` method, which handles Tensors' flow just like the Python version.


But there is also this `Impl` appended when we create the class. It's just an alias for a Shared-Pointer. We need this because the base-module needs to access the `DemoNet` class and the `torch::nn::Module` class.

Once we have created a class and appended the `Impl` to it, we need to pass this reference to `TORCH_MODULE`, shown in the last line.

```C++
// After the `DemoNet` class is created, we can create its object and use it for a forward pass.

// We create the output number of logits 
int64_t N = 10;
//instantite the model object
DemoNet net(N); //equivalent to `net = DemoNet(N)` in Pytorch

//create a random input and forward it.
torch::Tensor x = torch::randn({1,1,28,28});
torch::Tensor y = net->forward(x); // equivalent to `y = net(x)` in Pytorch

std::cout<<"Output shape from simple-CNN  is "<<y.sizes()<<std::endl;
```

### <font color='green'>3.2 The Sequential Way</font>

`nn.Sequential()` in PyTorch is meant for wrapping a series of operation; similarly, we can use `torch::nn::Sequential` in LibTorch.

The following is a simple example.

``` Python
## The Pytorch way.!
import torch.nn as nn
import torch
mod = nn.Sequential(nn.Conv2d(1, 32, 3, 1), 
                   nn.BatchNorm2d(32),
                   nn.Conv2d(32, 64, 3, 1),
                   nn.MaxPool2d(2,2))

logits = mod(torch.randn(3,1,28,28)) 
```

```C++
// The Libtorch way is 
torch::nn::Sequential simpleSequence(
    torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 32, 3).stride(1)),
    torch::nn::BatchNorm2d(32),  
    torch::nn::Conv2d(torch::nn::Conv2dOptions(32, 64, 3).stride(1)),
    torch::nn::MaxPool2d(torch::nn::MaxPool2dOptions(2).stride(2))
    );
            
// Calling the forward method on the Sequential object
torch::Tensor logits = simpleSequence->forward(torch::randn({3,1,28,28}));
std::cout<<"Output from sequential model is of size- "<<logits.sizes()<<std::endl;
```

**All the code shown above has been written in a C++ file. Let's run a few bash commands to execute that code. The following code cells will build and run the code in the  CPP file.**

In [1]:
%cd libtorch-intro/

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/libtorch-intro


In [2]:
!ls

CMakeLists.txt	Libtorch_Introduction.cpp  run_libtorch_basics.sh


In [3]:
!mkdir build

In [4]:
%cd build

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/libtorch-intro/build


In [5]:
! cmake  ..

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found Torch: /home/chetan/projects/piethon/pth_course/c3_w15_d

In [6]:
!cmake --build . --config Release

Scanning dependencies of target libtorch-basics
[ 50%] Building CXX object CMakeFiles/libtorch-basics.dir/Libtorch_Introduction.cpp.o
/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/libtorch-intro/Libtorch_Introduction.cpp:1:9: warning: #pragma once in main file
 #pragma once
         ^~~~
[100%] Linking CXX executable libtorch-basics
[100%] Built target libtorch-basics


In [7]:
%cd ..

/home/chetan/projects/piethon/pth_course/c3_w15_dl_pytorch/LibTorch/libtorch-intro


In [8]:
!./build/libtorch-basics

Shape of Tensor c is [2, 5, 3]
Shape of Tensor subA1 is [9, 2, 7, 2]
Residual sum between X1's subarray and Y1 after assignment is 0
Residual sum between X2's subarray and Y2's subarray after assignment is 0
Output shape from simple-CNN  is [1, 10]
Output from sequential model is of size- [3, 64, 12, 12]


## <font color='green'>References</font>

1. https://pytorch.org/cppdocs/index.html
2. https://github.com/pytorch/examples/blob/master/mnist/main.py